# 机器翻译和数据集


机器翻译（MT）：将一段文本从一种语言自动翻译为另一种语言，用神经网络解决这个问题通常称为神经机器翻译（NMT）。

主要特征：输出是单词序列而不是单个单词。 输出序列的长度可能与源序列的长度不同。

### 数据预处理

将数据集清洗、转化为神经网络的输入minbatch

字符在计算机里是以编码的形式存在，我们通常所用的空格是 \x20 ，是在标准ASCII可见字符 0x20~0x7e 范围内。
而 \xa0 属于 latin1 （ISO/IEC_8859-1）中的扩展字符集字符，代表不间断空白符nbsp(non-breaking space)，超出gbk编码范围，是需要去除的特殊字符。再数据预处理的过程中，我们首先需要对数据进行清洗。

### 分词

字符串---单词组成的列表

### 建立词典

单词组成的列表---单词id组成的列表

### 载入数据集

# Encoder-Decoder

 encoder：输入到隐藏状态  
 decoder：隐藏状态到输出


# Sequence to Sequence模型

### 模型：
训练  
![Image Name](https://cdn.kesci.com/upload/image/q5jc7a53pt.png?imageView2/0/w/640/h/640)
预测

![Image Name](https://cdn.kesci.com/upload/image/q5jcecxcba.png?imageView2/0/w/640/h/640)



### 具体结构：
![Image Name](https://cdn.kesci.com/upload/image/q5jccjhkii.png?imageView2/0/w/500/h/500)
### Encoder

# Decoder

### 损失函数

## Beam Search


## 机器翻译任务代码总结如下

### 数据预处理
1. 读取数据，处理数据中的编码问题，并将无效的字符串删除
2. 分词，分词的目的就是将字符串转换成单词组成的列表。目前有很多现成的分词工具可以直接使用，也可以直接按照空格进行分词(不推荐，因为分词不是很准确)
3. 建立词典，将单词组成的列表编程单词id组成的列表，这里会得到如下几样东西
   1. 去重后词典，及其中单词对应的索引列表
   2. 还可以得到给定索引找到其对应的单词的列表，以及给定单词得到对应索引的字典。
   3. 原始语料所有词对应的词典索引的列表
4. 对数据进行padding操作。因为机器翻译模型本质上是一个固定输入长度的Seq2Sqe模型，所以我们需要设置最大的数据长度，如果超出了设定的长度直接把后面的截断，少了的，根据需要进行不同的padding
    
    **数据预处理第四点，不对。基于rnn的Seq2Seq模型，可以处理任意长度，基于rnn的机器翻译也并不需要固定长度，关于这个你可以了解下“变长数据的读入”。
    要padding，是因为像tf、pytorch这些框架要求一个batch的数据必须长度相等，不然会报错；
    要截断，设置最大的数据长度是因为decode的时候达到这个长度我们就停止；再一个原因就是为了加快计算，不然的话为了单个特别长的数据，batch中的其他数据都补成这么长，多慢啊**

5. 制作数据生成器，但是需要注意的是对于翻译任务的数据格式，机器翻译的输入是一段文本序列，输出也是一段文本序列。

### Seq2Seq模型的构建
1. Seq2Seq模型由很多钟，但是整体框架都是基于先编码后解码的框架。也就是先对输入序列使用循环神经网络对他进行编码，编码成一个向量之后，再将编码得到的向量作为一个新的解码循环神经网络的隐藏状态的输入，进行解码，一次输出一个序列的元素，再将模型训练输出的序列元素与真实标签计算损失进行学习。
2. 词嵌入，一般情况下输入到编码网络中的数据不是一个onehot向量而是经过了编码之后的向量，比如由word2vec技术，让编码后的向量由更加丰富的含义。
3. 在进行编码和解码的过程中数据都是以时间步展开，也就是(Seq_len,)这种形式的数据进行处理的
4. 对于编码与解码的循环神经网络，可以通过控制隐藏层的层数及每一层隐藏层神经元的数量来控制模型的复杂度
5. 编码部分，RNN的用0初始化隐含状态，最后的输出主要是隐藏状态,编码RNN输出的隐含状态认为是其对应的编码向量
6. 解码器的整体形状与编码器是一样的，只不过解码器的模型的隐藏状态是由编码器的输出的隐藏状态初始化的。

### 损失函数
1. 解码器的输出是一个和词典维度相同的向量，其每个值对应与向量索引位置对应词的分数，一般是选择分数最大的那个词作为最终的输出。
2. 在计算损失函数之前，要把padding去掉，因为padding的部分不参与计算

### 测试
1. 解码器在测试的时候需要将模型的输出作为下一个时间步的输入
2. Beam Search搜索算法。
   1. 假设预测的时候词典的大小为3，内容为a,b,c. beam size为2，解码的时候过程如下
   2. 生成第一个词的时候，选择概率最大的两个词，假设为a,c.那么当前的两个序列就是a和c。
   3. 生成第二个词的时候，将当前序列a和c，分别与此表中的所有词进行组合，得到新的6个序列aa ab ac ca cb cc,计算每个序列的得分，并选择得分最高的2个序列，作为新的当前序列，假如为aa cb 
   4. 后面不断重复这个过程，直到遇到结束符或者达到最大长度为止，最终输出得分最高的2个序列。


# 习题

选择题

### 1.数据预处理中分词(Tokenization)的工作是？

A.把词语、标点用空格分开

B.把字符形式的句子转化为单词组成的列表

C.把句子转化为单词ID组成的列表

D.去除句子中的不间断空白符等特殊字符

**答案： B**

答案解释

选项二：正确，参考数据预处理分词(Tokenization)部分。

### 2.不属于数据预处理工作的是？

A.得到数据生成器

B.建立词典

C.分词

D.把单词转化为词向量

**答案：D**

答案解释

选项四：错误，单词转化为词向量是模型结构的一部分，词向量层一般作为网络的第一层。

### 3.下列不属于单词表里的特殊符号的是？

A.未知单词

B.空格符

C.句子开始符

D.句子结束符

**答案：B**

答案解释

选项二：错误，参考建立词典部分代码，空格不被认为是特殊字符，在该项目里，空格在预处理时被去除。

### 4.关于集束搜索(Beam Search)说法错误的是

A.集束搜索结合了greedy search和维特比算法。

B.集束搜索使用beam size参数来限制在每一步保留下来的可能性词的数量。

C.集束搜索是一种贪心算法。

D.集束搜索得到的是全局最优解。

**答案：D**

答案解释

选项一：正确，参考视频末尾Beam Search。

选项二：正确，参考视频末尾Beam Search。

选项三：正确，集束搜索是维特比算法的贪心形式。

选项四：错误，集束搜索是贪心算法。

### 5.不属于Encoder-Decoder应用的是

A.机器翻译

B.对话机器人

C.文本分类任务

D.语音识别任务

**答案：C**

答案解释

Encoder-Decoder常应用于输入序列和输出序列的长度是可变的，如选项一二四，而分类问题的输出是固定的类别，不需要使用Encoder-Decoder

### 6.关于Sequence to Sequence模型说法错误的是：

A.训练时decoder每个单元输出得到的单词作为下一个单元的输入单词。

B.预测时decoder每个单元输出得到的单词作为下一个单元的输入单词。

C.预测时decoder单元输出为句子结束符时跳出循环。

D.每个batch训练时encoder和decoder都有固定长度的输入。

**答案：A**

答案解释

选项一：错误，参考Sequence to Sequence训练图示。

选项二：正确，参考Sequence to Sequence预测图示。

选项三：正确，参考Sequence to Sequence预测图示。

选项四：正确，每个batch的输入需要形状一致。
